In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We've talked about Random Forests and Decision Trees, but now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 Lending data.

In [2]:
y2015 = pd.read_csv('./LoanStats3a.csv', skipinitialspace=True, header=1)

C:\Users\NutanMandale\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use get_dummies from pandas and see what happens off of this kind of naive approach.

In [ ]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)

cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN! You can restart the kernel in the 'Kernel' menu at the top of the page).

## Data Cleaning

Well, get_dummies can be a very memory intensive thing, particularly if things are typed poorly. We got a warning about that earlier. Mixed types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If theres a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all out categorical variables and see how many distinct counts there are...

In [3]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
3
term
2
int_rate
394
grade
7
sub_grade
35
emp_title
30659
emp_length
12
home_ownership
5
verification_status
3
issue_d
55
loan_status
4
pymnt_plan
1
desc
28964
purpose
14
title
21265
zip_code
837
addr_state
50
earliest_cr_line
530
revol_util
1119
initial_list_status
1
last_pymnt_d
112
next_pymnt_d
99
last_credit_pull_d
118
application_type
1


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [4]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(
                    y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line',
            'revol_util', 'sub_grade', 'addr_state', 'desc'],
            1, inplace=True)

# Remove two summary rows that don't actually contain data
y2015 = y2015[:-2]

Note that in `pd.to_numeric` we used the 'coerce' option for our conversion. This makes it so that if the function cannot figure out how to convert to numeric it returns a null for that value rather than returning an error for the whole process.

Now this should be better. Let's try again.

In [6]:
y2015 = y2015[:1000]

In [7]:
y2015.shape

(1000, 114)

In [8]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,last_credit_pull_d_Oct-2013,last_credit_pull_d_Oct-2014,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2012,last_credit_pull_d_Sep-2013,last_credit_pull_d_Sep-2014,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL
0,NaN,NaN,5000.0,5000.0,4975.000000,10.65,162.87,24000.00,27.65,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,NaN,NaN,2500.0,2500.0,2500.000000,15.27,59.83,30000.00,1.00,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,NaN,NaN,2400.0,2400.0,2400.000000,15.96,84.33,12252.00,8.72,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,NaN,NaN,10000.0,10000.0,10000.000000,13.49,339.31,49200.00,20.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,NaN,NaN,3000.0,3000.0,3000.000000,12.69,67.79,80000.00,17.94,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,NaN,NaN,5000.0,5000.0,5000.000000,7.90,156.46,36000.00,11.20,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,NaN,NaN,7000.0,7000.0,7000.000000,15.96,170.08,47004.00,23.51,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7,NaN,NaN,3000.0,3000.0,3000.000000,18.64,109.43,48000.00,5.35,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,NaN,NaN,5600.0,5600.0,5600.000000,21.28,152.39,40000.00,5.55,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
9,NaN,NaN,5375.0,5375.0,5350.000000,12.69,121.45,15000.00,18.08,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


It works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

In [9]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

array([ 0.97029703,  0.98019802,  0.99      ,  0.99      ,  0.99      ,
        0.97      ,  1.        ,  1.        ,  0.96969697,  1.        ])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us onluy 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matricies.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [10]:
# Your code here
from sklearn.decomposition import PCA

pca = PCA(n_components=4)
pca.fit(X)
PCA(copy=True, iterated_power='auto', n_components=4, random_state=None, svd_solver='auto', tol=0.0, whiten=False)
print(pca.explained_variance_ratio_)
X_reduced = pca.fit_transform(X)

cross_val_score(rfc, X_reduced, Y, cv=10)


[ 0.74423592  0.1785643   0.04647784  0.0144966 ]


array([ 0.98019802,  0.98019802,  0.93      ,  0.98      ,  0.94      ,
        0.99      ,  0.95      ,  0.96      ,  0.95959596,  0.92929293])

In [ ]:
# EXAMPLE SOLUTION

In [51]:
df = pd.DataFrame()
df['imp']= rfc.fit(X, Y).feature_importances_
df['features'] = X.columns

In [61]:
x = X.drop(['last_pymnt_amnt', 'out_prncp', 'last_pymnt_d_Jan-2017', 'out_prncp_inv'], 1)

In [62]:
cross_val_score(rfc, x, Y, cv=10)

array([ 0.97691814,  0.97967277,  0.98062264,  0.98059889,  0.97318927,
        0.97941107,  0.90975801,  0.97781947,  0.9503408 ,  0.95658576])

In [58]:
df.sort_values(by='imp', ascending=False)

,imp,features
24,1.566652e-01,last_pymnt_amnt
15,1.409654e-01,out_prncp
152,1.286368e-01,last_pymnt_d_Jan-2017
16,9.926417e-02,out_prncp_inv
167,7.363359e-02,next_pymnt_d_Feb-2017
182,6.249018e-02,last_credit_pull_d_Jan-2017
19,4.930034e-02,total_rec_prncp
147,2.987815e-02,last_pymnt_d_Dec-2016
17,2.503673e-02,total_pymnt
18,2.434818e-02,total_pymnt_inv
